<a href="https://colab.research.google.com/github/WVU-AIClub/Custom-TTS/blob/Sean-Branch/Piper/Piper_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

0) Colab runtime + GPU check

In [1]:
voice_folder = "Peter"
VOICE_NAME      = "PETAH"

# Check GPU
import torch, platform, sys
print("Python:", sys.version.split()[0])
print("PyTorch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
!nvidia-smi

Python: 3.10.12
PyTorch: 2.9.1+cu128
CUDA available: True
Wed Nov 19 15:17:47 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.95.05              Driver Version: 580.95.05      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4070 ...    Off |   00000000:01:00.0 Off |                  N/A |
| N/A   44C    P8              6W /   80W |      34MiB /   8188MiB |      0%      Default |
|                                         |                        |              

1) System packages (incl. eSpeak dev)

In [ ]:
# If not running on Google Colab. Run this yourself
!sudo apt-get update -y
!sudo apt-get install -y build-essential cmake ninja-build espeak-ng espeak-ng-data libespeak-ng-dev pkg-config ffmpeg
!pkg-config --modversion espeak-ng

[sudo] password for aiwvu: 


sudo: a password is required
^C
[sudo] password for aiwvu: 
sudo: a password is required
^C
Package espeak-ng was not found in the pkg-config search path.
Perhaps you should add the directory containing `espeak-ng.pc'
to the PKG_CONFIG_PATH environment variable
No package 'espeak-ng' found


2) Clone repo fresh

In [2]:
!rm -rf piper1-gpl
!git clone https://github.com/OHF-voice/piper1-gpl.git
%cd piper1-gpl
!pwd

Cloning into 'piper1-gpl'...
remote: Enumerating objects: 817, done.
remote: Counting objects: 100% (219/219), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 817 (delta 176), reused 157 (delta 155), pack-reused 598 (from 4)
Receiving objects: 100% (817/817), 4.69 MiB | 34.33 MiB/s, done.
Resolving deltas: 100% (455/455), done.
/home/aiwvu/Custom-TTS/piper1-gpl
/home/aiwvu/Custom-TTS/piper1-gpl


/home/aiwvu/Custom-TTS/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


3) Python deps (editable install, no venv in Colab)

In [ ]:
!python3 -m pip install --upgrade pip setuptools wheel
!python3 -m pip install -e ".[train]"

4) Build the Cython extension used for alignment

In [7]:
%cd piper1-gpl
!chmod +x ./build_monotonic_align.sh
!./build_monotonic_align.sh

[Errno 2] No such file or directory: 'piper1-gpl/'
/home/aiwvu/Custom-TTS/piper1-gpl
Compiling /home/aiwvu/Custom-TTS/piper1-gpl/src/piper/train/vits/monotonic_align/core.pyx because it changed.
[1/1] Cythonizing /home/aiwvu/Custom-TTS/piper1-gpl/src/piper/train/vits/monotonic_align/core.pyx
performance hint: core.pyx:5:0: Exception check on 'maximum_path_each' will always require the GIL to be acquired.
Possible solutions:
	1. Declare 'maximum_path_each' as 'noexcept' if you control the definition and you're sure you don't want the function to raise exceptions.
	2. Use an 'int' return type on 'maximum_path_each' to allow an error code to be returned.
performance hint: core.pyx:36:0: Exception check on 'maximum_path_c' will always require the GIL to be acquired.
Possible solutions:
	1. Declare 'maximum_path_c' as 'noexcept' if you control the definition and you're sure you don't want the function to raise exceptions.
	2. Use an 'int' return type on 'maximum_path_c' to allow an error co

5) Dev build (repo mode)

In [8]:
!python3 -m pip install --upgrade pip setuptools wheel scikit-build cmake ninja

  Using cached scikit_build-0.18.1-py3-none-any.whl.metadata (18 kB)
  Using cached cmake-4.1.2-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (6.5 kB)
  Using cached ninja-1.13.0-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (5.1 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached tomli-2.3.0-py3-none-any.whl.metadata (10 kB)
Using cached scikit_build-0.18.1-py3-none-any.whl (85 kB)
Using cached cmake-4.1.2-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (29.7 MB)
Using cached ninja-1.13.0-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (180 kB)
Using cached distro-1.9.0-py3-none-any.whl (20 kB)
Using cached tomli-2.3.0-py3-none-any.whl (14 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [scikit-build] [cmake]


In [ ]:
%cd /piper1-gpl
!python3 setup.py build_ext --inplace -v

7) Set paths and training hyperparams

In [ ]:
from pathlib import Path

# ==== CHANGE THESE ====

ESPEAK_VOICE    = "en-us"  # run `!espeak-ng --voices` to see options
SAMPLE_RATE_HZ  = 20250 # DEFAULT: 22050
BATCH_SIZE      = 32       # drop to 8 or 4 if you OOM

DATA_ROOT       = Path(f"{voice_folder}")
AUDIO_DIR       = DATA_ROOT / "wavs"
CSV_PATH        = DATA_ROOT / "metadata.csv"

CACHE_DIR       = Path(f"{voice_folder}/piper_cache")
CONFIG_PATH     = DATA_ROOT / f"{VOICE_NAME}.json"

# Optional: start from an existing checkpoint to speed up & stabilize training
# Get a .ckpt from https://huggingface.co/datasets/rhasspy/piper-checkpoints (medium quality recommended)
CKPT_PATH       = ""  # e.g., "/content/drive/MyDrive/piper_ckpts/en_US-lessac-medium.ckpt"

# Make sure dirs exist
CACHE_DIR.mkdir(parents=True, exist_ok=True)
print("CSV exists:", CSV_PATH.exists())
print("Audio dir exists:", AUDIO_DIR.exists())
print("Cache dir:", CACHE_DIR)
print("Config will be written to:", CONFIG_PATH)

8) Quick sanity checks

In [ ]:
!espeak-ng --voices | head -n 20

In [ ]:
import pandas as pd, os

csv_path = str(CSV_PATH)
if os.path.exists(csv_path):
    # Read as pipe-delimited, two columns
    try:
        df = pd.read_csv(csv_path, sep="|", header=None, names=["audio","text"])
        print(df.head())
        # Check a few audio files exist
        missing = [a for a in df["audio"].head(5) if not (AUDIO_DIR/str(a)).exists()]
        print("Missing among first 5:", missing)
    except Exception as e:
        print("CSV read error:", e)
else:
    print("CSV not found at:", csv_path)

9) Kick off training

In [ ]:
import subprocess
train_cmd = [
    sys.executable, "-m", 'piper.train fit',
  "--data.voice_name", f"{VOICE_NAME}",
  "--data.csv_path", f"/{voice_folder}/metadata.csv",
  "--data.audio_dir", f"/{voice_folder}/wavs",
  "--model.sample_rate", str(SAMPLE_RATE_HZ),
  "--data.espeak_voice", f"{ESPEAK_VOICE}",
  "--data.cache_dir", f"/{voice_folder}/piper_cache",
  "--data.config_path", f"/{voice_folder}/my_colab_voice.json",
  "--data.batch_size", str(BATCH_SIZE),
  "--ckpt_path", "https://huggingface.co/datasets/rhasspy/piper-checkpoints/resolve/main/en/en_US/lessac/medium/epoch%3D2164-step%3D1355540.ckpt"
]
subprocess.run(train_cmd, check=True)

10. Export to onnx

In [ ]:
# Checkpoint needs to be done dynamically
export_cmd = [
    sys.executable, "-m", "piper.train.export_onnx",
  "--checkpoint", "/piper1-gpl/lightning_logs/version_2/checkpoints/epoch=2174-step=2680.ckpt",
  "--output-file", f"/{voice_folder}/model.onnx"
]
subprocess.run(export_cmd, check=True)